# 03 – Feature Engineering for Traffy Fondue

ในสมุดบันทึกนี้ เราจะทำ **Feature Engineering** โดยนำข้อมูลจากหลายแหล่งมารวมกัน:

- ข้อมูล Traffy ที่ผ่านการทำความสะอาดแล้ว (`traffy_clean.csv`)
- ข้อมูลโรงพยาบาลในกรุงเทพ (scraped ด้วย BeautifulSoup)
- ข้อมูลสภาพอากาศรายชั่วโมงของกรุงเทพ (API จาก Open-Meteo)

## เป้าหมายหลัก

1. รวมข้อมูลภายนอก (external data) เข้ากับ Traffy ตาม *เขต* และ *เวลา*
2. สร้างฟีเจอร์ใหม่ เช่น  
   - `num_hospitals_in_district` (จำนวนโรงพยาบาลต่อเขต)  
   - `is_rainy_hour`, `rain_last_3h` (ฟีเจอร์จากฝน)  
   - `high_temperature` (ฟีเจอร์จากอุณหภูมิ)
3. บันทึกชุดข้อมูลสุดท้ายในรูปแบบ `traffy_features.csv` เพื่อใช้ในขั้นตอน ML (`04_ml_training.ipynb`)

> หมายเหตุ:  
> คอลัมน์ `resolution_time_hours` จะ **ไม่ถูกใช้เป็น feature** เพราะใช้สร้าง target `is_late` แล้ว  
> เราจะใช้เฉพาะฟีเจอร์ input ที่ไม่มี data leakage เท่านั้น


In [ ]:
import os
import logging

import pandas as pd
import numpy as np

from datetime import datetime, timedelta

BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))
DATA_DIR = os.path.join(BASE_DIR, "data")
EXTERNAL_DIR = os.path.join(DATA_DIR, "external")
FEATURE_DIR = os.path.join(DATA_DIR, "features")

os.makedirs(FEATURE_DIR, exist_ok=True)

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s",
)
logger = logging.getLogger("feature_engineering")

In [25]:
# 1) โหลด Traffy ที่ clean แล้ว
traffy_path = os.path.join(DATA_DIR, "bangkok_traffy_clean.csv")
df = pd.read_csv(traffy_path)

logger.info(f"Loaded Traffy clean data: {df.shape[0]} rows, {df.shape[1]} columns")
print(df.columns)

# 2) โหลดข้อมูลโรงพยาบาล
hosp_path = os.path.join(EXTERNAL_DIR, "bangkok_hospitals_network.csv")
df_hosp = pd.read_csv(hosp_path)
logger.info(f"Loaded hospital network data: {df_hosp.shape[0]} rows")

# 3) โหลดข้อมูลสภาพอากาศรายชั่วโมง
weather_path = os.path.join(EXTERNAL_DIR, "bangkok_hourly_weather_2023.csv")
df_weather = pd.read_csv(weather_path)
logger.info(f"Loaded weather data: {df_weather.shape[0]} rows")

2025-11-24 14:35:10,900 | Loaded Traffy clean data: 300002 rows, 15 columns
2025-11-24 14:35:10,903 | Loaded hospital network data: 109 rows
2025-11-24 14:35:10,909 | Loaded weather data: 8760 rows


Index(['type', 'organization', 'comment', 'district', 'timestamp', 'star',
       'count_reopen', 'month', 'lat', 'lon', 'hour', 'dayofweek', 'year',
       'resolution_time_hours', 'is_late'],
      dtype='object')


## 1) ทำความสะอาดคอลัมน์พื้นฐาน

สิ่งที่ต้องทำ:

- แปลง `timestamp` ให้เป็น datetime (จาก CSV มาจะเป็น string)
- ทำให้ชื่อคอลัมน์ `district` และ `district_th` มีรูปแบบมาตรฐานเดียวกัน  
  เช่น ตัดคำว่า "เขต", แปลงเป็นตัวพิมพ์เล็ก
- ตรวจดูชนิดข้อมูลของคอลัมน์ `hour`, `dayofweek`, `month`, `year` ว่าเป็น numeric แล้วหรือยัง

In [26]:
# จัดการชื่อคอลัมน์เล็กน้อย เผื่อมี space
df.columns = df.columns.str.strip()

# 1) แปลง timestamp เป็น datetime (timezone-naive พอสำหรับงานนี้)
df["timestamp"] = pd.to_datetime(
    df["timestamp"].astype(str).str.strip(),
    errors="coerce"
)

logger.info(f"timestamp dtype: {df['timestamp'].dtype}")
logger.info(f"NaT in timestamp: {df['timestamp'].isna().sum()} rows")

# ถ้ามีแถวที่ timestamp แปลงไม่ได้ ให้ทิ้งออก
df = df.dropna(subset=["timestamp"]).copy()
logger.info(f"After dropping NaT timestamp: {df.shape[0]} rows")

# 2) ฟังก์ชัน normalize ชื่อเขต (ตัดคำว่า 'เขต', lower-case)
def normalize_district(x):
    if pd.isna(x):
        return None
    return (
        str(x)
        .replace("เขต", "")
        .strip()
        .lower()
    )

df["district"] = df["district"].map(normalize_district)

# โรงพยาบาล: district_th น่าจะเป็นภาษาไทย
if "district_th" in df_hosp.columns:
    df_hosp["district_th"] = df_hosp["district_th"].map(normalize_district)
else:
    # เผื่อบางทีคอลัมน์ชื่ออื่น เช่น 'district'
    if "district" in df_hosp.columns:
        df_hosp["district_th"] = df_hosp["district"].map(normalize_district)
    else:
        raise RuntimeError("Cannot find district column in hospital dataset.")

df[["district"]].head()

2025-11-24 14:35:11,329 | timestamp dtype: datetime64[ns, UTC]
2025-11-24 14:35:11,330 | NaT in timestamp: 0 rows
2025-11-24 14:35:11,364 | After dropping NaT timestamp: 300002 rows


,district
0,None
1,ลาดพร้าว
2,ประเวศ
3,ลาดพร้าว
4,สาทร


## 2) สร้างฟีเจอร์จากข้อมูลโรงพยาบาล

แนวคิด:  
- เขตที่มีโรงพยาบาลมาก → มักเป็นเขตที่มีโครงสร้างพื้นฐานดี มีเจ้าหน้าที่และหน่วยงานรัฐเยอะ  
- เขตที่ไม่มี/มีน้อย → อาจเป็นพื้นที่ชานเมือง โครงสร้างพื้นฐานน้อย → โอกาส late สูงกว่า

เราจะสร้างฟีเจอร์:

- `num_hospitals_in_district` = จำนวนโรงพยาบาลในแต่ละเขต

แล้ว merge เข้าไปในตาราง Traffy ตามชื่อเขต

In [27]:
# นับจำนวนโรงพยาบาลต่อเขต
hosp_count = (
    df_hosp
    .groupby("district_th")
    .size()
    .reset_index(name="num_hospitals_in_district")
)

logger.info(f"Hospital counts per district: {hosp_count.shape[0]} districts")
hosp_count.head()

# merge เข้ากับ Traffy
df = df.merge(
    hosp_count,
    how="left",
    left_on="district",
    right_on="district_th"
)

# ถ้าเขตไหนไม่มีข้อมูลโรงพยาบาล ให้ใส่ 0
df["num_hospitals_in_district"] = df["num_hospitals_in_district"].fillna(0).astype(int)

# ไม่จำเป็นต้องเก็บ district_th ซ้ำ
df = df.drop(columns=["district_th"], errors="ignore")

df[["district", "num_hospitals_in_district"]].head()

2025-11-24 14:35:11,546 | Hospital counts per district: 41 districts


,district,num_hospitals_in_district
0,None,0
1,ลาดพร้าว,2
2,ประเวศ,4
3,ลาดพร้าว,2
4,สาทร,2


## 3) รวมข้อมูลสภาพอากาศ (Weather) ตามเวลา

โจทย์:  
- แต่ละ Traffy ticket มี `timestamp` (เวลาที่แจ้งปัญหา)  
- ข้อมูลอากาศมี `datetime` (รายชั่วโมง)  
→ เราจะจับ `timestamp` ปัดลงเป็น "ชั่วโมง" แล้ว join กับตาราง weather

ฟีเจอร์ที่จะได้ เช่น:

- `rain_mm` (ฝนตก mm ในชั่วโมงนั้น)
- `temperature` (อุณหภูมิ)
- `wind_speed`
- `is_rainy_hour` (ฝนมากกว่า threshold)
- `rain_last_3h` (ฝนสะสม 3 ชั่วโมงล่าสุด)

สภาพอากาศมีผลต่อความเร็วในการทำงานของเจ้าหน้าที่ → ช่วยโมเดลทำนาย late / not-late ได้ดีขึ้น

In [28]:
# --- Clean weather datetime ---
df_weather.columns = df_weather.columns.str.strip()
df_weather["datetime"] = pd.to_datetime(
    df_weather["datetime"].astype(str).str.strip(),
    errors="coerce"
)

logger.info(f"Weather datetime dtype: {df_weather['datetime'].dtype}")
logger.info(f"NaT in weather datetime: {df_weather['datetime'].isna().sum()} rows")

# --- Fix timezone mismatch ---
# Traffy timestamp is timezone-aware, so remove timezone
df["timestamp"] = df["timestamp"].dt.tz_localize(None)

# Weather datetime is naive (no timezone), but do tz_localize(None) anyway to ensure consistency
df_weather["datetime"] = df_weather["datetime"].dt.tz_localize(None)

# --- Round timestamp to hour ---
df["report_hour"] = df["timestamp"].dt.floor("h")   # use "h", not "H"

# --- Merge by hour ---
df = df.merge(
    df_weather,
    how="left",
    left_on="report_hour",
    right_on="datetime"
)

logger.info(f"After merging weather: {df.shape[0]} rows, {df.shape[1]} columns")

df[["timestamp", "report_hour", "datetime"]].head()

2025-11-24 14:35:11,701 | Weather datetime dtype: datetime64[ns]
2025-11-24 14:35:11,702 | NaT in weather datetime: 0 rows
2025-11-24 14:35:11,766 | After merging weather: 300002 rows, 21 columns


,timestamp,report_hour,datetime
0,2021-09-03 12:51:09.453003,2021-09-03 12:00:00,NaT
1,2021-12-13 05:53:36.861064,2021-12-13 05:00:00,NaT
2,2021-12-22 10:15:33.294829,2021-12-22 10:00:00,NaT
3,2021-12-09 12:29:08.408763,2021-12-09 12:00:00,NaT
4,2022-01-02 10:53:25.580723,2022-01-02 10:00:00,NaT


In [29]:
# เปลี่ยนชื่อ column จาก weather ให้เข้าใจง่าย
rename_map = {
    "precipitation": "rain_mm",
    "temperature_2m": "temperature",
    "wind_speed_10m": "wind_speed"
}
df = df.rename(columns={k: v for k, v in rename_map.items() if k in df.columns})

# ---------- สร้างฟีเจอร์จากสภาพอากาศ ----------

# ถ้าไม่มีฝนข้อมูล ให้แทน NaN ด้วย 0 สำหรับการคิด is_rainy_hour / rain_last_3h
df["rain_mm"] = df.get("rain_mm", 0)
df["rain_mm"] = df["rain_mm"].fillna(0)

# 1) is_rainy_hour: ชั่วโมงที่ฝน > 0.5 mm
df["is_rainy_hour"] = (df["rain_mm"] > 0.5).astype(int)

# 2) rain_last_3h: ฝนสะสม 3 ชั่วโมงล่าสุด
#    ต้อง sort ตามเวลา report_hour ก่อน
df = df.sort_values("report_hour")

# group ตามวันที่ (report_hour.dt.date) แล้ว rolling 3 ชั่วโมง
rain_rolling = (
    df.groupby(df["report_hour"].dt.date)["rain_mm"]
    .rolling(window=3, min_periods=1)
    .sum()
    .reset_index(level=0, drop=True)
)

df["rain_last_3h"] = rain_rolling

# 3) high_temperature: อุณหภูมิสูงผิดปกติ เช่น > 33°C
if "temperature" in df.columns:
    df["high_temperature"] = (df["temperature"] > 33).astype(int)
else:
    df["temperature"] = np.nan
    df["high_temperature"] = 0

# wind_speed ถ้าไม่มี ให้คอลัมน์ไว้ก่อน
if "wind_speed" not in df.columns:
    df["wind_speed"] = np.nan

df[["rain_mm", "is_rainy_hour", "rain_last_3h", "temperature", "high_temperature", "wind_speed"]].head()

,rain_mm,is_rainy_hour,rain_last_3h,temperature,high_temperature,wind_speed
0,0.0,0,0.0,NaN,0,NaN
3,0.0,0,0.0,NaN,0,NaN
1,0.0,0,0.0,NaN,0,NaN
2,0.0,0,0.0,NaN,0,NaN
4,0.0,0,0.0,NaN,0,NaN


## 4) รวมฟีเจอร์ทั้งหมดเป็นตารางเดียว

จากคอลัมน์ใน `traffy_clean.csv` + external features  
เราจะใช้ฟีเจอร์เหล่านี้เป็น input ของโมเดล ML:

- ข้อมูลพื้นฐานจาก Traffy
  - `type`, `organization`, `district`
  - `lat`, `lon`
  - `star`, `count_reopen`
  - `hour`, `dayofweek`, `month`, `year`
- ฟีเจอร์จากโรงพยาบาล
  - `num_hospitals_in_district`
- ฟีเจอร์จากสภาพอากาศ
  - `rain_mm`, `is_rainy_hour`, `rain_last_3h`
  - `temperature`, `high_temperature`, `wind_speed`
- target
  - `is_late` (0/1)

> หมายเหตุ:  
> - `resolution_time_hours` จะไม่อยู่ใน feature table เพราะใช้สร้าง target แล้ว  
> - การจัดการ missing values (imputation) จะไปทำในขั้นตอน ML pipeline (`04_ml_training.ipynb`)

In [30]:
feature_cols = [
    # categorical (จะ one-hot ในขั้น ML)
    "type",
    "organization",
    "district",

    # location
    "lat",
    "lon",

    # ticket behavior
    "star",
    "count_reopen",

    # time features
    "hour",
    "dayofweek",
    "month",
    "year",

    # infrastructure feature
    "num_hospitals_in_district",

    # weather features
    "rain_mm",
    "is_rainy_hour",
    "rain_last_3h",
    "temperature",
    "high_temperature",
    "wind_speed",

    # target
    "is_late",
]

# เลือกเฉพาะคอลัมน์ที่มีอยู่จริงใน df (กันกรณีบาง column ไม่มี)
feature_cols_existing = [c for c in feature_cols if c in df.columns]

df_features = df[feature_cols_existing].copy()
logger.info(f"Final feature table shape: {df_features.shape}")
df_features.head()

2025-11-24 14:35:12,400 | Final feature table shape: (300002, 19)


,type,organization,district,lat,lon,star,count_reopen,hour,dayofweek,month,year,num_hospitals_in_district,rain_mm,is_rainy_hour,rain_last_3h,temperature,high_temperature,wind_speed,is_late
0,{ความสะอาด},เขตบางซื่อ,None,100.53084,13.81865,0.0,0,12,4,9,2021,0,0.0,0,0.0,NaN,0,NaN,1
3,"{น้ำท่วม,ถนน}","เขตลาดพร้าว,ฝ่ายโยธา เขตลาดพร้าว",ลาดพร้าว,100.59165,13.82280,5.0,0,12,3,12,2021,2,0.0,0,0.0,NaN,0,NaN,1
1,{},"เขตลาดพร้าว,การไฟฟ้านครหลวง เขตนวลจันทร์",ลาดพร้าว,100.59131,13.80910,0.0,0,5,0,12,2021,2,0.0,0,0.0,NaN,0,NaN,1
2,{ท่อระบายน้ำ},"เขตประเวศ,ฝ่ายโยธา เขตประเวศ",ประเวศ,100.65440,13.68158,5.0,0,10,2,12,2021,4,0.0,0,0.0,NaN,0,NaN,1
4,"{ถนน,ทางเท้า}","เขตสาทร,ฝ่ายโยธา เขตสาทร",สาทร,100.53764,13.70716,0.0,0,10,6,1,2022,2,0.0,0,0.0,NaN,0,NaN,1


In [ ]:
# เซฟเป็น CSV (ใช้ CSV เป็นหลักอยู่แล้ว)
features_path = os.path.join(FEATURE_DIR, "traffy_features.csv")
os.makedirs(FEATURE_DIR, exist_ok=True)

df_features.to_csv(features_path, index=False, encoding="utf-8-sig")
logger.info(f"Saved feature table to: {features_path}")

2025-11-24 14:35:15,842 | Saved feature table to: /home/frostnzx/uniworks/datasci/final_project/data/features/traffy_features.csv


# Summary – Feature Engineering Layer Completed 

สิ่งที่ทำใน `03_feature_engineering.ipynb`:

1. โหลด `traffy_clean.csv` ที่ผ่านการทำความสะอาดในขั้นตอนก่อนหน้า  
2. รวมข้อมูลโรงพยาบาล (`bangkok_hospitals_network.csv`)  
   - สร้างฟีเจอร์ `num_hospitals_in_district`
3. รวมข้อมูลสภาพอากาศรายชั่วโมง (`bangkok_hourly_weather_2023.csv`)  
   - สร้างฟีเจอร์ `rain_mm`, `is_rainy_hour`, `rain_last_3h`, `temperature`, `high_temperature`, `wind_speed`
4. รวมทุกฟีเจอร์เข้าด้วยกัน พร้อม target `is_late`  
5. บันทึกเป็น `data/features/traffy_features.csv`  
   → ใช้เป็น input โดยตรงสำหรับโมเดลใน `04_ml_training.ipynb`

ขั้นตอนถัดไป:
- ไปที่ `04_ml_training.ipynb`
- โหลด `traffy_features.csv`
- สร้าง ML pipeline (imputer + one-hot + classifier)
- เทรนและประเมินโมเดล (`late vs not-late`)